# Матура БЕЛ: Топ 10 най-добре представящи се общини извън областните градове

# Тази тетрадка анализира матурите по БЕЛ

Първо извличаме данните чрез заявка в SPARQL от графова базата данни "semantic-schools". След това обработваме и почистваме данните и накрая анализираме в таблици.

In [ ]:
# Зареждане на библиотеки
import pandas as pd
import numpy as np
import os, io
import altair as alt

from google.colab import auth
import gspread
from google.auth import default

In [ ]:
%%capture
!pip3 install rdflib sparqlwrapper

from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF, CSV, TSV

# Функции, използвани в анализа

In [ ]:
def sub_to_eng(sub_bg):
    """Преобразуване на имената на предметите от български на английски"""
    if sub_bg == "БЕЛ":
        return "bel"
    elif sub_bg == "СТЕМ":
        return "stem"
    else:
        return "other"

def pup_weighted_average(x, var, weight_var="tot_pup_stem"):
    """ Изчисляване на претеглена средна на променлива.

    Args:
    x - pd.DataFrame. Масив от групирани данни, съдържащи променлива за осредняване и за претегляне
    var - string. Име на променлива за групиране
    weight_var - string. Име на променлива, с която данните да се претеглят. По подразбиране е tot_pup_stem

    Returns:
    weighted_average - int. Претеглена средна
    """
    x_clean = x[(x[var].notna()) & (x[weight_var].notna())]
    weighted_average = np.round_(
        np.average(x_clean[var], weights=x_clean[weight_var]), 2
        )
    return weighted_average

def average_var_geo(df_input, var, geo):
    """ Агрегиране на масив от данни, като се използва средната стойност на променлива,
    претеглена по броя ученици за дадено географско разделение

    Args:
    df_input - pd.DataFrame. Масив от данни за агрегиране
    var - string. Променлива за агрегиране
    geo - list of strings. Списък с имена на променливи, предоставящи географското разделение за агрегиране

    Returns:
    df_output - pd.DataFrame. Данни след агрегиране
    """
    df_output = df_input.groupby(geo).apply(
        pup_weighted_average, var=var
        ).reset_index().copy()
    # df_output.columns = geo + [var]
    df_output.columns = [geo, var]
    return df_output

def average_var_geo_year(df_input, var, w_var, geo):
    """ Агрегиране на масив от данни, като се използва средната стойност на променлива,
    претеглена по броя ученици за дадено географско разделение за всяка година, налична в данните

    Args:
    df_input - pd.DataFrame. Масив от данни за агрегиране
    var - string. Променлива за агрегиране
    w_var - string. Променлива с тегла
    geo - list of strings. Списък с имена на променливи, предоставящи географското разделение за агрегиране

    Returns:
    df_output - pd.DataFrame. Данни след агрегиране
    """
    df_output = df_input.groupby([geo, 'year']).apply(
        pup_weighted_average, var=var, weight_var=w_var
        ).reset_index().copy()
    df_output.columns = [geo, 'year', var]

    return df_output


# Извличане на данните чрез заявка в SPARQL от графовата база данни "semantic-schools"

In [ ]:
sparql = SPARQLWrapper("https://schools.ontotext.com/data/repositories/schools") #determine SPARQL endpoint

In [ ]:
query = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX : <https://schools.ontotext.com/data/resource/ontology/>
BASE <https://schools.ontotext.com/data/resource/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX schema: <http://schema.org/>
SELECT ?province ?town ?municipality ?school_wiki_id ?school ?year ?sub_group (SUM(?num_pup) AS ?tot_pup) (SUM(?grade*?num_pup)/?tot_pup AS ?average_grade)
  WHERE {
        ?s rdf:type qb:Observation;
           :date ?date;
           :grade_6 ?grade;
           :quantity_people ?num_pup;
           :school ?sch;
           :subject ?subject;
           :grade_level 12 .
        ?sch schema:name ?school;
             :wikidata_entity ?school_wiki_id;
             :place ?place.
        ?subject skos:prefLabel ?sub_abbr.
        ?place rdfs:label ?town;
                geo:sfWithin ?mun.
        ?mun geo:sfWithin ?prov;
             rdfs:label ?municipality.
        ?prov rdfs:label ?province.
        FILTER(lang(?town)="bg")
        FILTER(lang(?province)="bg")
    	FILTER(lang(?municipality)="bg")
        BIND (year(?date) as ?year)
        BIND (if (?sub_abbr in ("Математика"@bg, "Биология и здравно образование"@bg, "Физика и астрономия"@bg, "Химия и опазване на околната среда"@bg), "СТЕМ"@bg, if (?sub_abbr in ("Български език и литература"@bg), "БЕЛ"@bg, "ДРУГИ"@bg)) as ?sub_group)
} GROUP BY ?province ?town ?municipality ?school_wiki_id ?school ?year ?sub_group
ORDER BY ?province ?town ?school_wiki_id ?school ?year
"""

In [ ]:
sparql.setQuery(query)
sparql.setReturnFormat(CSV)   # Return format is JSON
results = sparql.query().convert()   # execute SPARQL query and write result to "matura_2021"

In [ ]:
df_maturi = pd.read_csv(io.StringIO(results.decode('utf-8')))

In [ ]:
df_maturi.head()

,province,town,municipality,school_wiki_id,school,year,sub_group,tot_pup,average_grade


In [ ]:
df_maturi.shape
# (15636, 8) от първоначалната оригинална заявка
# (15652, 9) след добавяне на колона за община

(15652, 9)

In [ ]:
df_maturi.municipality.unique()

array(['Община Хаджидимово', 'Община Банско', 'Община Белица',
       'Община Благоевград', 'Община Гоце Делчев', 'Община Сатовча',
       'Община Гърмен', 'Община Петрич', 'Община Кресна',
       'Община Симитли', 'Община Струмяни', 'Община Разлог',
       'Община Сандански', 'Община Якоруда', 'Община Айтос',
       'Община Бургас', 'Община Камено', 'Община Карнобат', 'Община Руен',
       'Община Малко Търново', 'Община Несебър', 'Община Поморие',
       'Община Приморско', 'Община Созопол', 'Община Средец',
       'Община Сунгурларе', 'Община Царево', 'Община Аврен',
       'Община Аксаково', 'Община Белослав', 'Община Бяла',
       'община Варна', 'Община Ветрино', 'Община Вълчи дол',
       'Община Девня', 'Община Долни чифлик', 'Община Дългопол',
       'Община Провадия', 'Община Суворово', 'Община Велико Търново',
       'Община Горна Оряховица', 'Община Елена', 'Община Златарица',
       'Община Стражица', 'Община Лясковец', 'Община Павликени',
       'Община Полски Тръмбеш', '

In [ ]:
# замяна на райони от Столична община със "Столична община"
# речник със стойности
replace_dict = {'Банкя (район)': 'Столична община',
                'Район Панчарево': 'Столична община',
                'Район Кремиковци': 'Столична община',
                'Нови Искър (район)': 'Столична община'}

# замяна на стойности
df_maturi = df_maturi.replace(replace_dict)

In [ ]:
df_maturi.municipality.nunique()

241

# Обработка и почистване на данни

### Завъртане на данните

In [ ]:
matura_clean = df_maturi.copy()

Преименуване на предметите от български на английски

In [ ]:
matura_clean["sub_eng"] = [sub_to_eng(x) for x in matura_clean["sub_group"]]

Разделяне на броя на учениците и средния успех по групи предмети

In [ ]:
matura_clean = matura_clean.pivot(
    index=["province", "municipality", "town", "school_wiki_id", "school", "year"],
    columns="sub_eng",
    values=["tot_pup", "average_grade"],
).reset_index()
matura_clean.columns = matura_clean.columns.map("_".join).str.strip("_")

In [ ]:
matura_clean = matura_clean[
    [
        "province",
        "municipality",
        "town",
        "school_wiki_id",
        "school",
        "year",
        "tot_pup_bel",
        "tot_pup_other",
        "tot_pup_stem",
        "average_grade_bel",
        "average_grade_other",
        "average_grade_stem"
    ]
]

### Премахване на дублирани наблюдения за едно и също училище

In [ ]:
"Има {} реда в данните, докато уникалните комбинации година-училище са {}".format(matura_clean.shape[0],
                                                                         len(matura_clean[["year", "school_wiki_id"]].drop_duplicates()))

'Има 6212 реда в данните, докато уникалните комбинации година-училище са 5654'

In [ ]:
matura_clean["school_instances"] = matura_clean.groupby(["year", "school_wiki_id"])["school"].transform(len)

In [ ]:
matura_clean["school_instances"].value_counts(sort="ascending")

1    5096
2    1116
Name: school_instances, dtype: int64

Някои училища в София се дублират два пъти, защото фигурират в два града (София и квартал, напр. Младост).

In [ ]:
matura_clean["province"].value_counts(sort="ascending")

Област София Град        1356
Област Пловдив            420
Област Варна              307
Област Благоевград        294
Област Бургас             289
Област Плевен             269
Област Стара Загора       228
Софийска област           222
Област Пазарджик          218
Област Велико Търново     218
Област Враца              190
Област Русе               183
Област Шумен              161
Област Добрич             157
Област Монтана            153
Област Кърджали           151
Област Хасково            147
Област Смолян             140
Област Сливен             137
Област Ловеч              120
Област Разград            111
Област Кюстендил          110
Област Силистра           108
Област Ямбол              107
Област Перник             104
Област Видин               99
Област Търговище           94
Област Габрово             84
Столична община            35
Name: province, dtype: int64

In [ ]:
matura_clean[matura_clean["school_instances"]>=2]["province"].value_counts(sort="ascending")

Област София Град      1062
Област Пазарджик         12
Столична община          11
Област Благоевград        6
Област Бургас             6
Област Перник             6
Област Стара Загора       6
Област Кюстендил          5
Област Враца              1
Област Монтана            1
Name: province, dtype: int64

In [ ]:
matura_clean[(matura_clean["school_instances"]>=2) & (matura_clean["province"]=="Област София Град")]["town"].value_counts(sort="ascending")

София                      528
Младост (район в София)     54
Люлин (район)               45
Оборище                     36
Средец                      36
Слатина (район)             30
Подуяне                     30
Триадица                    30
Лозенец                     30
Възраждане                  30
Красна поляна (район)       27
Надежда                     24
Район Красно Село           24
Сердика                     23
Овча купел (район)          18
Илинден (район)             18
Изгрев (район)              18
Столична община             18
Искър (район)               16
Връбница                    12
Витоша (район)              12
Студентски (район)           3
Name: town, dtype: int64

Всички училища в София, които не се дублират, са записани със стойност "София" в колоната town, така че могат да бъдат запазени само наблюденията със стойност "София" в колоната town.

In [ ]:
matura_clean[(matura_clean["school_instances"]==1) & (matura_clean["province"]=="Област София Град")]["town"].value_counts()

София    294
Name: town, dtype: int64

Запазване само наблюденията със стойност "София" в колоната town.

In [ ]:
matura_clean = matura_clean.loc[~((matura_clean["province"]=="Област София Град") & (matura_clean["town"]!="София"))].copy()

Останалите дубликати изглежда се дължат на имена на градове/селища, които съществуват в няколко области.

In [ ]:
matura_clean["school_instances"] = matura_clean.groupby(["year", "school_wiki_id"])["school"].transform(len)

In [ ]:
matura_clean[matura_clean["school_instances"]>=2]["province"].value_counts(sort="ascending")

Област Пазарджик       12
Област Благоевград      6
Област Бургас           6
Област Перник           6
Област Стара Загора     6
Област Кюстендил        5
Столична община         5
Област Враца            1
Област Монтана          1
Name: province, dtype: int64


След ръчна проверка премахваме грешки в данните.

In [ ]:
matura_clean = matura_clean.loc[~(((matura_clean["school_wiki_id"]=="http://www.wikidata.org/entity/Q106347968")
                                  & (matura_clean["province"]!="Област Бургас"))
                                 |((matura_clean["school_wiki_id"]=="http://www.wikidata.org/entity/Q106348023")
                                  & (matura_clean["province"]!="Област Стара Загора"))
                                 |((matura_clean["school_wiki_id"]=="http://www.wikidata.org/entity/Q95950966")
                                  & (matura_clean["province"]!="Област Кюстендил"))
                                 |((matura_clean["school_wiki_id"]=="http://www.wikidata.org/entity/Q106348020")
                                  & (matura_clean["province"]!="Област Монтана")))].copy()

In [ ]:
matura_clean["school_instances"] = matura_clean.groupby(["year", "school_wiki_id"])["school"].transform(len)

In [ ]:
matura_clean.shape[0] == len(matura_clean[["year", "school_wiki_id"]].drop_duplicates())

False

In [ ]:
matura_clean.shape[0] # 5660

5660

In [ ]:
len(matura_clean[["year", "school_wiki_id"]].drop_duplicates()) # 5654

5654

## Изчистване на липсващи наблюдения

In [ ]:
matura_clean.apply(lambda x: sum(x.isna()))

province                  0
municipality              0
town                      0
school_wiki_id            0
school                    0
year                      0
tot_pup_bel               9
tot_pup_other           852
tot_pup_stem           1945
average_grade_bel         9
average_grade_other     852
average_grade_stem     1945
school_instances          0
dtype: int64

В малка част от наблюденията липсват резултатите по БЕЛ, което е странно, тъй като всички ученици би трябвало да имат матура по БЕЛ.

In [ ]:
matura_clean[matura_clean["tot_pup_bel"].isna()]

,province,municipality,town,school_wiki_id,school,year,tot_pup_bel,tot_pup_other,tot_pup_stem,average_grade_bel,average_grade_other,average_grade_stem,school_instances
40,Област Благоевград,Община Благоевград,Благоевград,http://www.wikidata.org/entity/Q106329170,"Седмо средно училище ""Кузман Шапкарев""",2021,NaN,NaN,1.0,NaN,NaN,2.44,1
126,Област Благоевград,Община Гърмен,Гърмен,http://www.wikidata.org/entity/Q106346434,"Средно училище ""Св.Св.Кирил и Методий""",2020,NaN,NaN,1.0,NaN,NaN,4.50,1
348,Област Бургас,Община Бургас,Бургас,http://www.wikidata.org/entity/Q106346962,"Средно училище ""Христо Ботев""",2021,NaN,2.0,NaN,NaN,2.73,NaN,1
391,Област Бургас,Община Бургас,Бургас,http://www.wikidata.org/entity/Q106349374,Професионална гимназия по морско корабоплаване...,2020,NaN,1.0,NaN,NaN,3.53,NaN,1
909,Област Велико Търново,Община Велико Търново,Велико Търново,http://www.wikidata.org/entity/Q106348314,"Средно училище ""Георги Стойков Раковски""",2020,NaN,1.0,NaN,NaN,4.07,NaN,1
1085,Област Велико Търново,Община Стражица,Камен (Област Велико Търново),http://www.wikidata.org/entity/Q106347078,"Средно училище ""Св. Климент Охридски""",2019,NaN,2.0,NaN,NaN,3.34,NaN,1
4913,Област София Град,Столична община,София,http://www.wikidata.org/entity/Q62104785,"55 СРЕДНО УЧИЛИЩЕ ""ПЕТКО КАРАВЕЛОВ""",2017,NaN,1.0,NaN,NaN,3.32,NaN,1
4972,Област София Град,Столична община,София,http://www.wikidata.org/entity/Q92188738,"17 СУ ""Дамян Груев""",2023,NaN,6.0,NaN,NaN,2.26,NaN,1
5348,Област Стара Загора,Община Стара Загора,Стара Загора,http://www.wikidata.org/entity/Q106346541,"Средно училище ""Христо Ботев""",2021,NaN,1.0,NaN,NaN,2.84,NaN,1


Заместване на липсващите наблюдения в променливите за общ брой ученици с 0

In [ ]:
tot_pup_vars = [x for x in list(matura_clean.columns) if x.find("tot_pup")==0]
matura_clean[tot_pup_vars]=matura_clean[tot_pup_vars].fillna(0)

In [ ]:
matura_clean_wide = matura_clean.pivot(
    index=["province", "municipality", "town", "school_wiki_id", "school"],
    columns="year",
    values=['tot_pup_bel', 'tot_pup_other', 'tot_pup_stem',
            'average_grade_bel', 'average_grade_other', 'average_grade_stem'],
).reset_index().copy()
matura_clean_wide.columns = matura_clean_wide.columns.map(lambda x: "_".join([str(y) for y in x])).str.strip("_")
matura_clean_wide[0:5]

,province,municipality,town,school_wiki_id,school,tot_pup_bel_2017,tot_pup_bel_2019,tot_pup_bel_2020,tot_pup_bel_2021,tot_pup_bel_2022,...,average_grade_other_2020,average_grade_other_2021,average_grade_other_2022,average_grade_other_2023,average_grade_stem_2017,average_grade_stem_2019,average_grade_stem_2020,average_grade_stem_2021,average_grade_stem_2022,average_grade_stem_2023
0,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106347222,"Средно училище ""Неофит Рилски""",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.190000,NaN,NaN,NaN,NaN,NaN,NaN
1,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106348341,"Гимназия по туризъм ""Алеко Константинов""",17.0,22.0,18.0,4.0,16.0,...,NaN,NaN,4.020000,3.000000,NaN,NaN,3.1100,NaN,NaN,NaN
2,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106349506,"Професионална лесотехническа гимназия""Никола Й...",33.0,39.0,33.0,45.0,36.0,...,3.265,NaN,3.890000,3.800000,4.730000,3.150000,3.5600,NaN,NaN,NaN
3,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106349516,Професионална гимназия по електроника и енерге...,37.0,52.0,28.0,52.0,31.0,...,4.090,4.762222,5.146552,NaN,4.190000,3.946154,3.5700,5.358,NaN,NaN
4,Област Благоевград,Община Белица,Белица,http://www.wikidata.org/entity/Q106346288,"Средно училище ""Св.Св.Кирил и Методий""",58.0,32.0,37.0,51.0,40.0,...,3.860,4.470909,4.217692,3.183415,4.782308,3.986667,4.3225,4.360,NaN,NaN


## Проверка на училищата, за които няма налични данни за всички години

Като цяло броят на училищата изглежда намалява с времето.

In [ ]:
matura_clean["year"].value_counts()
# 2017    950
# 2023    945
# 2019    941
# 2021    941
# 2020    940
# 2022    937

2017    951
2023    946
2019    942
2021    942
2020    941
2022    938
Name: year, dtype: int64

На ниво училище, обаче, картината изглежда малко по-сложна от това.

In [ ]:
matura_clean["school_instances"] = matura_clean.groupby(["school_wiki_id"])["school"].transform(len)

In [ ]:
"{} училища присъстват за по-малко от 5 години.".format(len(matura_clean[matura_clean["school_instances"]<5]["school_wiki_id"].drop_duplicates()))

'52 училища присъстват за по-малко от 5 години.'

In [ ]:
matura_clean[matura_clean["school_instances"]<5].groupby(["province","school_wiki_id"])["year"].apply(lambda x: ",".join(x.astype("string")))[0:10]

province            school_wiki_id                           
Област Благоевград  http://www.wikidata.org/entity/Q106329263         2020,2021,2023
                    http://www.wikidata.org/entity/Q106346885         2021,2022,2023
                    http://www.wikidata.org/entity/Q106347222                   2023
Област Бургас       http://www.wikidata.org/entity/Q106329068    2017,2019,2020,2021
                    http://www.wikidata.org/entity/Q106346962    2017,2019,2020,2021
                    http://www.wikidata.org/entity/Q106347174              2022,2023
                    http://www.wikidata.org/entity/Q106348041    2017,2019,2020,2021
Област Варна        http://www.wikidata.org/entity/Q106329135              2020,2023
                    http://www.wikidata.org/entity/Q106346146                   2023
                    http://www.wikidata.org/entity/Q106346437         2021,2022,2023
Name: year, dtype: object

# Анализ

Създаване на масив от данни с общ брой ученици по област и предмет за всяка година

In [ ]:
tot_pup_vars = [x for x in list(matura_clean.columns) if x.find("tot_pup")==0]
matura_province = matura_clean.groupby(["province", "year"])[tot_pup_vars].aggregate(sum).reset_index()

In [ ]:
matura_province[0:5]

,province,year,tot_pup_bel,tot_pup_other,tot_pup_stem
0,Област Благоевград,2017,2425.0,413.0,811.0
1,Област Благоевград,2019,2571.0,1111.0,572.0
2,Област Благоевград,2020,2322.0,912.0,505.0
3,Област Благоевград,2021,2423.0,1138.0,466.0
4,Област Благоевград,2022,2300.0,1965.0,247.0


Създаване на допълнителна True/False колона, за да се премахнат съвпаденията между област и община


In [ ]:
matura_clean['not_province'] = matura_clean.apply(lambda x: x.town in x.province, axis=1)
# matura_clean['not_province'] = matura_clean.apply(lambda x: x.municipality in x.province, axis=1)
matura_town = matura_clean[matura_clean['not_province'] == False] #премахване на областните градове
matura_town.head()

,province,municipality,town,school_wiki_id,school,year,tot_pup_bel,tot_pup_other,tot_pup_stem,average_grade_bel,average_grade_other,average_grade_stem,school_instances,not_province
0,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106347222,"Средно училище ""Неофит Рилски""",2023,20.0,20.0,0.0,4.52,4.19,NaN,1,False
1,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106348341,"Гимназия по туризъм ""Алеко Константинов""",2017,17.0,2.0,0.0,3.59,3.15,NaN,6,False
2,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106348341,"Гимназия по туризъм ""Алеко Константинов""",2019,22.0,2.0,0.0,3.20,4.38,NaN,6,False
3,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106348341,"Гимназия по туризъм ""Алеко Константинов""",2020,18.0,0.0,1.0,3.15,NaN,3.11,6,False
4,Област Благоевград,Община Банско,Банско,http://www.wikidata.org/entity/Q106348341,"Гимназия по туризъм ""Алеко Константинов""",2021,4.0,0.0,0.0,3.51,NaN,NaN,6,False


Извличане на топ 10 най-високи средни резултати по година и община

In [ ]:
matura_town_mean = matura_town.groupby(['municipality'],as_index=False)['average_grade_bel'].mean()
matura_top10 = matura_town_mean.nlargest(10,'average_grade_bel').reset_index()
top10_per_year = matura_town.groupby('year').apply(lambda x: x.nlargest(10, 'average_grade_bel')).reset_index(drop=True)
top10_per_year[['municipality','year','average_grade_bel']]

,municipality,year,average_grade_bel
0,Община Козлодуй,2017,5.32
1,Община Казанлък,2017,5.09
2,Община Разлог,2017,5.02
3,Община Карлово,2017,5.02
4,Община Свищов,2017,5.01
5,Община Челопеч,2017,4.97
6,Община Баните,2017,4.95
7,Община Карнобат,2017,4.93
8,Община Дупница,2017,4.93
9,Община Пирдоп,2017,4.93


Визуализиране на топ 10 най-високи резултати по години

In [ ]:
domain = [2017,2018,2019,2020,2021,2022,2023]
alt.Chart(top10_per_year).mark_point().encode(
    alt.X(
        'average_grade_bel:Q',
        title="Средна оценка на матура по БЕЛ",
        axis=alt.Axis(grid=False),
        scale=alt.Scale(domain=[4, 5.5])
    ),
    alt.Y(
        'municipality:N',
        title="",
        sort='x',
        axis=alt.Axis(grid=True, labelLimit=450),
    ),
    color=alt.Color('year:N', scale=alt.Scale(domain=domain), legend=alt.Legend(title="Година")),
    tooltip=['municipality', 'year', 'average_grade_bel']
).properties(
    height=alt.Step(20)
).configure_view(stroke="transparent")

alt.Chart(...)

Създаване на таблица за обозначение и сортиране на общините, които най-често са попадали в топ 10

In [ ]:
mun_counts = top10_per_year['municipality'].value_counts()
ordered_mun= pd.DataFrame({'Value': mun_counts.index, 'Frequency': mun_counts.values})
ordered_mun= ordered_mun.sort_values(by='Frequency', ascending=False).reset_index(drop = True)
ordered_mun

,Value,Frequency
0,Община Карлово,6
1,Община Казанлък,6
2,Община Челопеч,5
3,Община Козлодуй,4
4,Община Разлог,4
5,Община Панагюрище,3
6,Община Златоград,3
7,Община Велинград,3
8,Община Божурище,3
9,Община Неделино,2


Извличане на топ 10 най-добре представящи се общини според сумата от средните оценки през годините (за да се разграничат общините, които се срещат еднакъв брой пъти в топ 10 за съответната година)

In [ ]:
unique_values = top10_per_year['municipality'].unique()

data_list = []

for mun in unique_values:
  filtered_rows = top10_per_year[top10_per_year['municipality'] == mun]
  sum_of_values = filtered_rows['average_grade_bel'].sum()
  data_list.append({'Value': mun, 'Sum': sum_of_values})

top10_mun = pd.DataFrame(data_list)
top10_mun = top10_mun.sort_values(by='Sum', ascending=False).reset_index(drop=True)
top10_mun = top10_mun.head(10)
top10_mun



,Value,Sum
0,Община Казанлък,29.63
1,Община Карлово,29.26
2,Община Челопеч,24.84
3,Община Козлодуй,20.34
4,Община Разлог,19.55
5,Община Божурище,14.91
6,Община Панагюрище,14.89
7,Община Велинград,14.49
8,Община Златоград,14.26
9,Община Свищов,10.07


In [ ]:

alt.Chart(top10_mun).mark_circle().encode(
    alt.X(
        'Sum:Q',
        title="",
        axis=alt.Axis(grid=False),
        scale=alt.Scale(domain=[8, 30])
    ),
    alt.Y(
        'Value:N',
        title="",
        sort='x',
        axis=alt.Axis(grid=True, labelLimit=450),
    ),
    size=alt.Size('Sum:Q', scale=alt.Scale(range=[10, 500])),
    color=alt.Color('Sum:Q'),
    tooltip=['Value','Sum']
).properties(
    height=alt.Step(30)
).configure_view(stroke="transparent")

alt.Chart(...)

# Извличане на данни

In [ ]:
# len(matura_clean)
matura_clean.to_csv("извличам_общини.csv", index=False)